<a href="https://colab.research.google.com/github/Michal-Janulewski/Uczenie-Maszynowe-w-Finansach/blob/Quiz-1/Zadanie_1_Uczenie_maszynowe_w_finansach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, auc, classification_report

In [55]:
#Pobieramy dane z githuba
data = pd.read_csv("https://raw.githubusercontent.com/Michal-Janulewski/Uczenie-Maszynowe-w-Finansach/Quiz-1/case1.csv", sep = ";")

In [56]:
data = data.drop(["Index"], axis=1)
data = data.drop(["Address"], axis=1)
data = data.drop(["Unnamed: 0"], axis=1)

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
# numeryczne
numericals = data.select_dtypes(include=['float','int']).columns
data[numericals].describe()

In [ ]:
categories = data.select_dtypes('O').columns.astype('category')
data[categories]

In [61]:
data.drop(data[categories], axis=1, inplace=True)

In [ ]:
data[numericals].var()

In [ ]:
print(data['FLAG'].value_counts())

In [ ]:
# Filtering the features with 0 variance
var0 = data.var() == 0
print(data.var()[var0])
print('\n')
# Drop features with 0 variance --- these features will not help in the performance of the model
data.drop(data.var()[var0].index, axis = 1, inplace = True)
print(data.var())
print(data.shape)

In [ ]:
#Sprawdzamy czy dane mają brakujące wartości
data.isnull().sum()

In [ ]:
# Zastąpienie brakujących zmiennych medianą
data.fillna(data.median(), inplace=True)
data.isnull().sum()

In [ ]:
#Sprawdzamy czy istnieją zduplikowane wartości
data[data.duplicated()]

In [68]:
#Usuwamy zduplikowane wartości
data.drop_duplicates(keep="first",inplace=True)

In [ ]:
# Macierz korealcji
corr = data.corr()

mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)]=True
with sns.axes_style('white'):
    fig, ax = plt.subplots(figsize=(30,30))
    sns.heatmap(corr,  mask=mask, annot=True, cmap='CMRmap', center=0, linewidths=0.1, square=True)

In [ ]:
data.info()

In [71]:
# Drop one of those highly correlated features
data.drop(['total transactions (including tnx to create contract','total ether sent contracts', 'max val sent to contract','max val sent', 'avg value sent to contract', 'Unique Sent To Addresses','Unique Received From Addresses', 'total ether received',  'min value received', 'min val sent', 'Total ERC20 tnxs', 'ERC20 uniq rec addr', 'ERC20 uniq rec contract addr', 'ERC20 min val rec', 'ERC20 max val rec', 'ERC20 avg val rec', 'ERC20 min val sent', 'ERC20 max val sent', 'ERC20 avg val sent' ], axis=1, inplace=True)

In [ ]:
data.info()

In [ ]:
y = data.iloc[:, 0]
X = data.iloc[:, 1:]
print(X.shape, y.shape)

In [ ]:
# Podział na zbiór treningowy (80%) i zbiór testowy (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [75]:
PT = PowerTransformer()
PT_train = PT.fit_transform(X_train)
PT_test = PT.transform(X_test)

In [ ]:
PT_df = pd.DataFrame(PT_train, columns=X_train.columns)
PT_df

In [77]:
oversample = SMOTE()
print(f'Shape of the training before SMOTE: {PT_df.shape, y_train.shape}')

x_tr_resample, y_tr_resample = oversample.fit_resample(PT_df, y_train)
print(f'Shape of the training after SMOTE: {x_tr_resample.shape, y_tr_resample.shape}')

Shape of the training before SMOTE: ((7430, 19), (7430,))
Shape of the training after SMOTE: ((12174, 19), (12174,))


In [ ]:
LR = LogisticRegression(random_state=123)
LR.fit(x_tr_resample, y_tr_resample)


norm_test_f = PT.transform(X_test)

preds = LR.predict(norm_test_f)

In [ ]:
print(classification_report(y_test, preds))
print(confusion_matrix(y_test, preds))

In [87]:
accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
confusion_matrix(y_test, preds)

Accuracy: 0.8611410118406889
Precision: 0.5533980582524272
Recall: 0.9105431309904153


array([[1315,  230],
       [  28,  285]])

In [ ]:
#RandomForest Classifier
RF = RandomForestClassifier(random_state=123)
model2= RF.fit(x_tr_resample, y_tr_resample)

# Transform test features
sc_test = PT.transform(X_test)

pred2 = model2.predict(sc_test)

accuracy = accuracy_score(y_test, pred2)
precision = precision_score(y_test, pred2)
recall = recall_score(y_test, pred2)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
confusion_matrix(y_test, pred2)

In [ ]:
print(classification_report(y_test, pred2))

In [ ]:
#KNeighbors Classifier
knn = KNeighborsClassifier(n_neighbors=3)
model3= knn.fit(x_tr_resample, y_tr_resample)
pred3 = model3.predict(X_test)

accuracy = accuracy_score(y_test, pred3)
precision = precision_score(y_test, pred3)
recall = recall_score(y_test, pred3)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
confusion_matrix(y_test, pred3)

In [ ]:
print(classification_report(y_test, pred3))

In [ ]:
#Ostatecznie wybór padł na model klasyfikujący według drzew losowych, ponieważ w tym modelu było najmniej przypadków gdy model klasyfikował przypadki nie będace fraudami jako fraudy (1514 z 1545 zsotało poprawnie zaklasyfikowanych)
#W przypadku dwóch pozostałych modeli błędnie zaklasyfikowanych obiektów było znacznie więcej.